#### Preprocess the transportation assumptions and outputs from the GCAM-USA simulations
You can skip these steps if using the pre-processed files provided in the input directory

In [ ]:
import gcamreader
import numpy as np
import pandas as pd

In [ ]:
# GCAM transportation assumptions files; prefer values in first file
# the GCAM-USA source code referemced here can be downloaded from https://doi.org/10.5281/zenodo.7838871
GCAM_path = './GCAMUSA_GODEEEP'
transportation_assumption_files = [
    f'{GCAM_path}/input/gcamdata/inst/extdata/energy/OTAQ_trn_data_EMF37.csv',
    f'{GCAM_path}/input/gcamdata/inst/extdata/energy/UCD_trn_data_CORE.csv',
]
transportation_assumptions = [pd.read_csv(f, comment='#') for f in transportation_assumption_files]

In [ ]:
# merge both assumptions files but prefer values in the first file (based on guidance from GCAM-USA team)
merged = transportation_assumptions[0][
    transportation_assumptions[0].UCD_region == 'USA'
].merge(
    transportation_assumptions[1][
        transportation_assumptions[1].UCD_region == 'USA'
    ],
    how='outer',
    on=['UCD_region', 'UCD_sector', 'mode', 'size.class', 'UCD_technology', 'UCD_fuel', 'variable'],
    suffixes=('_primary', '_secondary')
)
primary = merged[[c for c in merged.columns if not '_secondary' in c]]
primary.columns = primary.columns.str.replace('_primary', '')
secondary = merged[[c for c in merged.columns if not '_primary' in c]]
secondary.columns = secondary.columns.str.replace('_secondary', '')
assumptions = primary.combine_first(secondary)

# write assumptions to excel file
assumptions.to_excel('./input/transportation_assumptions_godeeep_ira_ccs_climate.xlsx', index=False)

In [ ]:
# get the service output from GCAM-USA
# the GCAM-USA databases used here can be downloaded from https://doi.org/10.5281/zenodo.7838871
#for gcam_database, scenario_name in [('database_01b_BAU_climate', 'business_as_usual_ira_ccs_climate'), ('database_02b_NZ_climate', 'net_zero_ira_ccs_climate')]:
for gcam_database, scenario_name in [('Sensitivity_database_basexdb_NetZeroCCS_Climate', 'net_zero_ccs_climate')]:
    gcam_db_path = '/Users/thur961/godeeep'
    gcam_db_file = f'{gcam_database}'
    query_path = './input/queries.xml'
    conn = gcamreader.LocalDBConn(gcam_db_path, gcam_db_file)
    queries = gcamreader.parse_batch_query(query_path)
    service_output = conn.runQuery(queries[0])
    energy_output = conn.runQuery(queries[1])
    # write to excel files
    service_output.to_csv(f'./input/transportation_service_output_godeeep_{scenario_name}.csv', index=False)
    energy_output.to_csv(f'./input/transportation_energy_output_godeeep_{scenario_name}.csv', index=False)

In [ ]:
# SCRATCH

In [ ]:
nz_service = pd.read_csv('./input/transportation_service_output_godeeep_NetZeroNoCCS_Climate.csv')
nz_energy = pd.read_csv('./input/transportation_energy_output_godeeep_NetZeroNoCCS_Climate.csv')
bau_service = pd.read_csv('./input/transportation_service_output_godeeep_BAU_Climate.csv')
bau_energy = pd.read_csv('./input/transportation_energy_output_godeeep_BAU_Climate.csv')
nz_ira_service = pd.read_csv('./input/transportation_service_output_godeeep_net_zero_ira_ccs_climate.csv')
nz_ira_energy = pd.read_csv('./input/transportation_energy_output_godeeep_net_zero_ira_ccs_climate.csv')
bau_ira_service = pd.read_csv('./input/transportation_service_output_godeeep_business_as_usual_ira_ccs_climate.csv')
bau_ira_energy = pd.read_csv('./input/transportation_energy_output_godeeep_business_as_usual_ira_ccs_climate.csv')

In [ ]:
nz_service_LDV = nz_service[(nz_service.Year >= 2020) & nz_service.technology.isin(['BEV']) & nz_service.subsector.isin(['2W and 3W', 'Car', 'Large Car and Truck', 'Light truck'])].groupby(['subsector', 'Year']).aggregate({'value': 'sum'})
nz_service_LDV

In [ ]:
nz_ira_service_LDV = nz_ira_service[(nz_ira_service.Year >= 2020) & nz_ira_service.technology.isin(['BEV']) & nz_ira_service.subsector.isin(['2W and 3W', 'Car', 'Large Car and Truck', 'Light truck'])].groupby(['subsector', 'Year']).aggregate({'value': 'sum'})
nz_ira_service_LDV